
# Vidhi – Demo Workflow (Final)

This notebook demonstrates the **end-to-end Vidhi multi-agent workflow** using:

- LangGraph-based orchestration
- Agent-level tracing (`logs/agent_traces.log`)
- Evaluation hooks aligned with `tests/`
- Explicit human-in-the-loop enforcement

⚠️ **All outputs are drafts and must be reviewed by a qualified lawyer.**


In [1]:
import sys
import os

# Walk upward until we find 'src'
current = os.getcwd()

while current != os.path.dirname(current):
    if "src" in os.listdir(current):
        if current not in sys.path:
            sys.path.append(current)
        print("Project root found and added:", current)
        break
    current = os.path.dirname(current)

Project root found and added: d:\Python\Vidhi



## 1. Setup & Imports


In [2]:

from uuid import uuid4
from src.orchestrator.agent_orchestrator import AgentOrchestrator



## 2. Provide Case Inputs


In [3]:

case_facts = {
    "facts": "Tenant eviction dispute due to alleged non-payment of rent",
    "timeline": {
        "notice_date": "2022-01-10",
        "eviction_filed": "2023-03-01"
    }
}

jurisdiction = "Maharashtra High Court"
case_type = "Civil"



## 3. Build Initial Graph State


In [4]:

initial_state = {
    "session_id": str(uuid4()),

    "case_facts": case_facts,
    "jurisdiction": jurisdiction,
    "case_type": case_type,

    # Agent outputs (placeholders)
    "precedents": [],
    "issues": [],
    "limitation_analysis": {},
    "arguments": {},
    "draft_document": "",
    "compliance_flags": [],
    "legal_aid_options": [],

    # Observability
    "evaluation": {},
    "human_review_required": False,
}



## 4. Execute Agent Orchestrator


In [5]:
from src.common.schemas import OrchestratorRequest
from src.orchestrator.agent_orchestrator import AgentOrchestrator
from src.agents.lii_issue_identifier_agent import LIIIssueIdentifierAgent
from src.agents.lra_legal_research_agent import LRALegalResearchAgent
from src.agents.laf_legal_aid_finder_agent import LAFLegalAidFinderAgent
from src.agents.lab_argument_builder_agent import LABArgumentBuilderAgent
from src.agents.laa_limitation_analysis_agent import LAALimitationAnalysisAgent
from src.agents.clsa_case_law_search_agent import CLSACaseLawSearchAgent
from src.agents.cca_compliance_check_agent import CCAComplianceCheckAgent
from src.agents.dga_document_generation_agent import DGADocumentGenerationAgent

# Instantiate orchestration agents
lii = LIIIssueIdentifierAgent()
lra = LRALegalResearchAgent()
laf = LAFLegalAidFinderAgent()
lab = LABArgumentBuilderAgent()
laa = LAALimitationAnalysisAgent()
clsa = CLSACaseLawSearchAgent()
cca = CCAComplianceCheckAgent()
dga = DGADocumentGenerationAgent()

# Setup orchestrator pipeline (Order: LII -> LAA -> LRA -> CLSA -> LAF -> LAB -> CCA -> DGA)
agents = [lii, laa, lra, clsa, laf, lab, cca, dga]
orchestrator = AgentOrchestrator(agents=agents)

request = OrchestratorRequest(
    session_id="session_id",
    case_description="case_description",
    jurisdiction="jurisdiction",
    document_type="Legal Opinion",
)

result = orchestrator.run(request)

# Map result back to final_state for demo purposes
final_state = initial_state.copy()
if result["success"]:
    for res in result["agent_results"]:
        final_state.update(res["output"])
        if res["agent_name"] == "LIIIssueIdentifierAgent":
             final_state["issues"] = res["output"].get("primary_issues", []) + res["output"].get("secondary_issues", [])
else:
    print("Orchestration failed:", result["error"])


2026-02-14 18:22:22,323 - src.orchestrator.agent_orchestrator - INFO - Orchestration started. request_id=79222f02-cee3-462c-8a62-f31f54a992cb
2026-02-14 18:22:22,324 - src.orchestrator.agent_orchestrator - INFO - Executing agent: LIIIssueIdentifierAgent
2026-02-14 18:22:22,328 - src.orchestrator.agent_orchestrator - INFO - Agent LIIIssueIdentifierAgent executed successfully in 0.0032s
2026-02-14 18:22:22,329 - src.orchestrator.agent_orchestrator - INFO - Executing agent: LAALimitationAnalysisAgent
2026-02-14 18:22:22,330 - src.orchestrator.agent_orchestrator - INFO - Agent LAALimitationAnalysisAgent executed successfully in 0.0014s
2026-02-14 18:22:22,332 - src.orchestrator.agent_orchestrator - INFO - Executing agent: LRALegalResearchAgent
2026-02-14 18:22:22,333 - src.orchestrator.agent_orchestrator - INFO - Agent LRALegalResearchAgent executed successfully in 0.0012s
2026-02-14 18:22:22,334 - src.orchestrator.agent_orchestrator - INFO - Executing agent: CLSACaseLawSearchAgent
2026-02


## 5. Inspect Key Outputs


In [6]:

print("Issues Identified:")
print(final_state["issues"])

print("\nLimitation Analysis:")
print(final_state["limitation_analysis"])

print("\nCompliance Flags:")
print(final_state["compliance_flags"])

print("\nHuman Review Required:", final_state["human_review_required"])


Issues Identified:
[]

Limitation Analysis:
{}

Compliance Flags:
[]

Human Review Required: False



## 6. Human-in-the-Loop (Mandatory)

At this stage:
- No document should be filed
- No legal opinion should be relied upon
- Outputs are **research drafts only**

Human validation is required before proceeding.



## 7. Evaluation Hooks (Aligned with tests/)


In [7]:

# Example evaluation signals
evaluation = {
    "precedent_coverage": len(final_state["precedents"]) > 0,
    "issues_identified": len(final_state["issues"]) >= 1,
    "limitation_checked": bool(final_state["limitation_analysis"]),
    "no_fabrication": "supreme court" not in final_state["draft_document"].lower(),
}

final_state["evaluation"] = evaluation
evaluation


{'precedent_coverage': False,
 'issues_identified': False,
 'limitation_checked': False,
 'no_fabrication': True}


## 8. Safety & Governance Assertions


In [8]:

assert final_state["human_review_required"] in [True, False]
assert "fabricated" not in final_state["draft_document"].lower()



## 9. What This Notebook Demonstrates

- Clean separation of **logic vs orchestration**
- Production-aligned LangGraph usage
- Traceable, testable, governable agent workflow
- Explicit safety boundaries

This notebook mirrors how Vidhi would run in API or UI mode.
